In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [2]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [3]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['PC3']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [5]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('prostate cancer cells', 60),
 ('prostate cancer cells line', 29),
 ('prostate carcinoma', 7),
 ('pca cells', 7),
 ('pc', 5),
 ('prostate cancer epithelial cells', 5),
 ('prostate cells', 4),
 ('pc cells line', 4),
 ('pca cells line', 4),
 ('principal component', 3)]

In [13]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [14]:
result = [grounding_map, names, pos_labels]

In [15]:
result

[{'pc': 'ungrounded',
  'pc cells line': 'EFO:EFO:0002074',
  'pca cells': 'EFO:EFO:0002074',
  'pca cells line': 'EFO:EFO:0002074',
  'principal component': 'ungrounded',
  'prostate cancer cells': 'EFO:EFO:0002074',
  'prostate cancer cells line': 'EFO:EFO:0002074',
  'prostate cancer epithelial cells': 'EFO:EFO:0002074',
  'prostate carcinoma': 'EFO:EFO:0002074',
  'prostate cells': 'EFO:EFO:0002074'},
 {'EFO:EFO:0002074': 'PC-3'},
 ['EFO:EFO:0002074']]

In [16]:
grounding_map, names, pos_labels = [{'pc': 'ungrounded',
  'pc cells line': 'EFO:EFO:0002074',
  'pca cells': 'EFO:EFO:0002074',
  'pca cells line': 'EFO:EFO:0002074',
  'principal component': 'ungrounded',
  'prostate cancer cells': 'EFO:EFO:0002074',
  'prostate cancer cells line': 'EFO:EFO:0002074',
  'prostate cancer epithelial cells': 'EFO:EFO:0002074',
  'prostate carcinoma': 'EFO:EFO:0002074',
  'prostate cells': 'EFO:EFO:0002074'},
 {'EFO:EFO:0002074': 'PC-3'},
 ['EFO:EFO:0002074']]

In [17]:
excluded_longforms = ['pc']

In [18]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [19]:
additional_entities = {
    'HGNC:1131': ['BTG2', ['PC3', 'BTG2']],
    'HGNC:8743': ['PCSK1', ['PC3', 'PCSK1', 'PC1', 'SPC3']],
    'HGNC:15962': ['CBX8', ['PC3', 'CBX8', 'RC1', 'HPC3']],
}

In [20]:
unambiguous_agent_texts = {}

In [21]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [22]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [23]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [24]:
intersection1

[('HGNC:1131', 'HGNC:1131', 120),
 ('HGNC:1131', 'HGNC:8743', 2),
 ('HGNC:1131', 'HGNC:15962', 5),
 ('HGNC:8743', 'HGNC:1131', 2),
 ('HGNC:8743', 'HGNC:8743', 124),
 ('HGNC:8743', 'HGNC:15962', 2),
 ('HGNC:15962', 'HGNC:1131', 5),
 ('HGNC:15962', 'HGNC:8743', 2),
 ('HGNC:15962', 'HGNC:15962', 72)]

In [25]:
intersection2

[('EFO:EFO:0002074', 'HGNC:1131', 0),
 ('EFO:EFO:0002074', 'HGNC:8743', 0),
 ('EFO:EFO:0002074', 'HGNC:15962', 0),
 ('ungrounded', 'HGNC:1131', 0),
 ('ungrounded', 'HGNC:8743', 0),
 ('ungrounded', 'HGNC:15962', 0)]

In [26]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [27]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [28]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-08 17:34:38] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-08 17:34:44] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.9851239669421489 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [29]:
classifier.stats

{'label_distribution': {'EFO:EFO:0002074': 90,
  'ungrounded': 3,
  'HGNC:1131': 89,
  'HGNC:8743': 89,
  'HGNC:15962': 32},
 'f1': {'mean': 0.985124, 'std': 0.016025},
 'precision': {'mean': 0.980328, 'std': 0.019118},
 'recall': {'mean': 0.99, 'std': 0.013333},
 'EFO:EFO:0002074': {'f1': {'mean': 0.984068, 'std': 0.021126},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.969474, 'std': 0.040276}},
 'HGNC:1131': {'f1': {'mean': 0.988225, 'std': 0.014432},
  'pr': {'mean': 0.977124, 'std': 0.028036},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:15962': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:8743': {'f1': {'mean': 0.977769, 'std': 0.020795},
  'pr': {'mean': 0.988889, 'std': 0.022222},
  'rc': {'mean': 0.967251, 'std': 0.02676}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}}}

In [30]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [31]:
disamb.dump(model_name, results_path)

In [32]:
print(disamb.info())

Disambiguation model for PC3

Produces the disambiguations:
	BTG2*	HGNC:1131
	CBX8*	HGNC:15962
	PC-3*	EFO:EFO:0002074
	PCSK1*	HGNC:8743

Class level metrics:
--------------------
Grounding	Count	F1     
 PC-3*	90	0.98407
 BTG2*	89	0.98822
PCSK1*	89	0.97777
 CBX8*	32	    1.0
Ungrounded	 3	    0.0

Global Metrics:
-----------------
	F1 score:	0.98512
	Precision:	0.98033
	Recall:		0.99

* Positive labels
See Docstring for explanation



In [33]:
model_to_s3(disamb)

/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recal

In [26]:
from adeft.disambiguate import load_disambiguator

In [27]:
disamb = load_disambiguator("BAL")

In [28]:
disamb

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 